<a href="https://colab.research.google.com/github/ParkSlingshot/-/blob/main/Textclassfication_TFhub_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Ready 
import tensorflow as tf
from tensorflow import keras
!pip install -q tensorflow -hub
!pip install -q tfds -nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense
print('GPU', 'On' if tf.config.experimental.list_physical_devices("GPU") else 'off')

In [ ]:
#Data setting
train_data, validation_data, test_data = tfds.load(
    name = 'imdb_reviews', split = ('train[:60%]', 'train[60%:]', 'test'), as_supervised = True
)

In [ ]:
train_examples_batch, train_labels_bathc = next(iter(train_data.batch(10)))

#Modeling

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype = tf.string, trainable=True)
hub_layer (train_examples_batch[:5])
model = keras.Sequential()
model.add(hub_layer)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512))

In [ ]:
loss, acc = model.evaluate(test_data.batch(512))
print('Loss : %.5f  Accuracy : %.5f' %(loss, acc))